In [2]:
import json
import pandas as pd
import os
import numpy as np
import fiona
import folium
import geopandas as gpd
import random
import asf_search as asf
from shapely.geometry import Polygon
from shapely.geometry import mapping

workspace_directory = r'C:\Users\trygg\Documents\Master_project' #HOME
os.chdir(workspace_directory)# Change the current working directory to the specified workspace


### Create the big dictionary

Create a dictionary where each unique granule has information about what measurements correspond to it's footprint and date are added as a as a line in the dictionary.

Flow:

Go through every ASF item which has been date filtered and grabs the granule footprint.

For each item, it checks the entire Vindefjallen data set to test if the XY coords are located within the granule footprint,
and add relevant columns to the dataset if conditions are true.

If there is a match, then it is added to the dictionary.

The end result is a dictinary with a list of every granule that has at least one measurement within the footprint and other relevant information regarding the measurements.

In [4]:
#Vindefjallen data
#import the excel file with Vindefjallen corner coordinates and measurements.
csv_file_path_2023 = r"Data\Vindefjallen_data\Vindefjallen_cleaning\Vindefjallen_Granule_processing\vindefjallen_corner_coords_processing_2023_middletime.csv"
df_measurements = pd.read_csv(csv_file_path_2023) # Read the CSV file into a DataFrame
vindefjallen_triangleCorners_numpy_2023 = df_measurements.to_numpy()# Convert the DataFrame to a NumPy array for easier coding
df_measurements.head()

,ID,OG_ID,Triangle,Corner,TriangleCorner,date,SnowDepth,SnowDepth_Noplus,temperature,x_4326,y_4326,x_3006,y_3006,MiddleTime
0,0,1,AC-42,A,AC-42A,2017-02-21,50,50,-5.0,15.760981,65.908407,534661.4522,7309911.059,10:50:00
1,1,2,AC-38,A,AC-38A,2017-02-26,50,50,-10.0,16.028612,65.896595,546872.4418,7308768.327,08:55:00
2,2,3,AC-37,A,AC-37A,2017-02-26,45,45,-8.0,15.848811,65.938003,538617.1852,7313260.876,11:27:30
3,3,4,AC-31,A,AC-31A,2017-02-26,20,20,-8.0,15.595766,65.984498,527055.8412,7318310.670,13:52:30
4,4,5,AC-39,A,AC-39A,2017-03-04,60,60,-15.0,15.936407,65.881455,542696.2178,7307015.189,09:35:00


In [5]:
def Point_in_polygon(x, y, polygon):
    """Check if a point (x, y) is inside a polygon defined by a list of coordinates.
    Parameters:
    - x, y: Coordinates of the point to be checked.
    - polygon: List of tuples representing the vertices of the polygon.
    xxx ChatGPT"""
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                if p1x == p2x or x <= xinters:
                    inside = not inside
        p1x, p1y = p2x, p2y
    return inside

#### ASF

https://docs.asf.alaska.edu/asf_search/ASFSearchResults/

https://docs.asf.alaska.edu/asf_search/searching/

https://github.com/asfadmin/Discovery-asf_search/blob/405dcd6efe822ca08efdc24dd39d67849220101e/examples/1-Basic_Overview.ipynb

In [7]:
import asf_search as asf
#my_env home

#study area polygon
aoi = 'POLYGON((15.5702 66.3423,15.4488 66.3354,15.4242 66.2903,15.346 66.2493,15.2874 66.2273,15.2277 66.2165,15.145 66.1704,15.1619 66.0775,15.2024 66.0619,15.3474 66.0313,15.5304 65.9813,15.6281 65.9312,15.63 65.8965,15.741 65.8534,15.8627 65.8514,16.0288 65.8348,16.1754 65.8159,16.2049 65.8012,16.339 65.7877,16.4512 65.8283,16.2466 65.864,16.121 65.9472,16.1321 66.0205,16.2592 66.0296,16.4784 65.9773,16.6739 65.9642,16.8497 65.9721,16.9572 66.0108,16.7932 66.0502,16.5999 66.0381,16.4575 66.0817,16.2843 66.1399,16.2622 66.1561,15.9537 66.251,15.5702 66.3423))'
opts_slc = {
    'platform': asf.PLATFORM.SENTINEL1,
    'beamMode': asf.BEAMMODE.IW,
    'start': '2017-01-01T01:00:00Z',
    'end': '2018-06-11T23:59:59Z',
    'processingLevel' : asf.PRODUCT_TYPE.SLC #SLC datatype
}
asf_coords_slc = asf.geo_search(intersectsWith=aoi, **opts_slc)
print(f'{len(asf_coords_slc)} results found - SLC')

823 results found - SLC


##### ASF SLC dictionary

In [8]:
# make a list of every granule that has measurements covering the study area and study period
ordabok_granule_id_slc = {}
teljari = 0
for asf_coord in asf_coords_slc:
    polygon = asf_coord.geometry['coordinates'][0]
    measurment_in_granule_ID = []
    measurment_in_granule_cornerTriangle = []
    measurment_in_granule_date = []
    measurment_in_granule_depth = []
    measurment_in_granule_temperature = []

    # for corner in vindefjallen_triangleCorners_numpy_celcius: #Old
    for corner in vindefjallen_triangleCorners_numpy_2023:
        point = (corner[9], corner[10])  # x, y coordinates from the excel file
        start_time_match = corner[5] == asf_coord.properties['startTime'][:10]

        if Point_in_polygon(point[0], point[1], polygon) and start_time_match:
            measurment_in_granule_ID.append(corner[0])
            measurment_in_granule_date.append(corner[5])
            measurment_in_granule_depth.append(corner[6])
            measurment_in_granule_cornerTriangle.append(corner[4])
            measurment_in_granule_temperature.append(corner[8])

    if measurment_in_granule_date:
        ordabok_granule_id_slc[teljari] = {
            'granule_ID': asf_coord.properties['sceneName'],
            'orbitProp': asf_coord.properties['flightDirection'],
            'pathNr': asf_coord.properties['pathNumber'],
            'frameNr': asf_coord.properties['frameNumber'],
            'date': asf_coord.properties['startTime'][:10],
            'startTime': asf_coord.properties['startTime'],
            'measurement_date': measurment_in_granule_date[0],
            'measurement_id': measurment_in_granule_ID,
            'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
            'measurement_depth': measurment_in_granule_depth,
            'measurement_temperature': measurment_in_granule_temperature,
            'nr_of_measurements': len(measurment_in_granule_ID),
            'polygon': polygon
        }
        teljari += 1

print('length:', len(ordabok_granule_id_slc))

length: 20


In [ ]:
# inverts the diciotnary created in the above cell so each measurement now has the granule id attach to it
ordabok_granule_id = ordabok_granule_id_slc
def granule_every_point_dict_func(ordabok_granule_id):
    """
    Create a dictionary of granule data for each point based on matching corner triangles and dates.

    Parameters:
    ordabok_granule_id (dict): A dictionary containing granule data.

    Returns:
    pandas.DataFrame: A DataFrame containing the granule data for each point.

    """
    counter = 0
    granule_every_point_dict = {}
    for i in ordabok_granule_id:
        for j in range(len(ordabok_granule_id[i]['measurement_cornerTriangle'])):
            for k in vindefjallen_triangleCorners_numpy_2023:
                if ordabok_granule_id[i]['measurement_cornerTriangle'][j] == k[4] and ordabok_granule_id[i]['measurement_date'] == k[5]:
                    granule_every_point_dict[counter] = {
                        'granule_id': ordabok_granule_id[i]['granule_ID'],
                        'date': ordabok_granule_id[i]['measurement_date'],
                        'measurement_id': ordabok_granule_id[i]['measurement_id'][j],
                        'snow_depth': ordabok_granule_id[i]['measurement_depth'][j],
                        'temperature': ordabok_granule_id[i]['measurement_temperature'][j],
                        'corner': ordabok_granule_id[i]['measurement_cornerTriangle'][j],
                        'x_coord': k[9],
                        'y_coord': k[10],
                        'x_coord3006': k[11],
                        'y_coord3006': k[12]
                    }
                    counter += 1
    granule_df = pd.DataFrame.from_dict(granule_every_point_dict, orient='index')
    return granule_df

slc_granule_every_point_dict_df = granule_every_point_dict_func(ordabok_granule_id_slc)